<a href="https://colab.research.google.com/github/royen2000/yolov8/blob/main/yolov8_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

yolov8 lib install

In [8]:
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install ultralytics

yolov8 lib install

In [3]:
import torch
from ultralytics import YOLO

# YOLOv8 모델 로드
model = YOLO("yolov8n.pt")  # 또는 다른 버전의 모델을 사용할 수 있습니다.

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 327MB/s]


In [ ]:
import cv2

# 이미지 로드
img = cv2.imread('image_path_here.jpg')

# 이미지 크기 조정 (필요한 경우)
img = cv2.resize(img, (640, 480))  # 예시 크기

# 모델에 입력할 수 있는 형태로 변환
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [10]:
# 객체 인식
results = model(img_rgb)

# 결과 출력
results.print()
results.show()  # 결과 이미지를 표시합니다.



0: 480x640 1 truck, 8.4ms
Speed: 2.0ms preprocess, 8.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)


AttributeError: 'list' object has no attribute 'print'

In [5]:
# 결과에서 특정 클래스 인식
for result in results.xyxy[0]:
    if result[-1] in [2, 0, 1, 7]:  # COCO 데이터셋 클래스 인덱스: 사람(0), 오토바이(1), 트럭(7), 신호등(9) 참고
        x, y, x2, y2, conf, cls = result
        print(f"Class: {model.names[int(cls)]}, Confidence: {conf:.2f}")
        cv2.rectangle(img, (int(x), int(y)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, model.names[int(cls)], (int(x), int(y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'results' is not defined

In [31]:
import torch
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

# YOLOv8 모델 로드
model = YOLO("yolov8s.pt")

# 이미지 로드 및 전처리
image_path = "/content/sample_data/mnist_train_small.csv"  # 예시 이미지 경로
img = cv2.imread(image_path)

if img is None:
    raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {image_path}")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 객체 인식 수행
results = model(img_rgb)

# 결과 검사
if len(results) == 0 or not results[0].boxes:
    raise ValueError("모델 결과가 없습니다. 입력 이미지를 확인하세요.")

# 필터링할 클래스: 신호등(9), 사람(0), 트럭(7), 오토바이(1) - COCO 데이터셋 클래스 인덱스
target_classes = [0, 1, 7, 9]

# 바운딩 박스 데이터로 시각화
for result in results[0].boxes:
    # 바운딩 박스 좌표(x1, y1, x2, y2), 신뢰도(conf), 클래스(cls) 정보 얻기
    x1, y1, x2, y2 = result.xyxy[0].tolist()  # 바운딩 박스 좌표
    conf = result.conf[0].item()  # 신뢰도
    cls = int(result.cls[0].item())  # 클래스 인덱스

    # YOLO 모델의 클래스 이름에 접근하는 방법 수정
    if cls < len(model.names):  # 유효한 클래스 인덱스 범위 체크
        print(f"클래스 인덱스: {cls}, 클래스 이름: {model.names[cls]}, 신뢰도: {conf:.2f}")

        # 라벨 생성 (클래스 이름과 신뢰도)
        label = f"{model.names[cls]} {conf:.2f}"

        # 바운딩 박스 그리기
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        print(f"유효하지 않은 클래스 인덱스: {cls}")

# Matplotlib로 결과 시각화
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("YOLO Detection - Filtered Classes")
plt.show()


FileNotFoundError: 이미지를 찾을 수 없습니다: /content/sample_data/mnist_train_small.csv